In [3]:
import requests
import time
import hmac
import hashlib
from urllib.parse import urlencode
import os
from dotenv import load_dotenv
from datetime import datetime, timedelta

# 加载 .env 文件中的环境变量
dotenv_path = '.env'
load_dotenv(dotenv_path)

# 从环境变量中获取 API Key 和 API Secret
api_key = os.getenv('api_key')
api_secret = os.getenv('api_secret')

def hashing(query_string):
    return hmac.new(
        api_secret.encode("utf-8"), query_string.encode("utf-8"), hashlib.sha256
    ).hexdigest()

# 起始时间（3个月前）
start_time = int((datetime.now() - timedelta(days=90)).timestamp() * 1000)
# 当前时间
end_time = int(datetime.now().timestamp() * 1000)

data = []
total_data_points = 0

while start_time < end_time:
    params = {
        'symbol': '1000PEPEUSDC',
        'interval': '1m',  # 1分钟线
        'startTime': start_time,
        'limit': 1500,
        'timestamp': int(time.time_ns() / 10**6)
    }
    params['signature'] = hashing(urlencode(params))

    response = requests.get('https://fapi.binance.com/fapi/v1/klines', params=params, headers={
        'X-MBX-APIKEY': api_key
    })

    if response.status_code == 200:
        klines = response.json()
        if not klines:
            break
        
        data.extend(klines)
        total_data_points += len(klines)
        
        # 打印进度
        latest_time = datetime.fromtimestamp(klines[-1][6] / 1000)
        print(f"Fetched {len(klines)} data points, total: {total_data_points}, latest time: {latest_time}")

        # 更新start_time为上次返回数据的最后一根K线的结束时间
        start_time = klines[-1][6] + 1
    else:
        print(f"Error: {response.status_code}, {response.text}")
        break

# 打印或处理数据
print(f"Total data points fetched: {total_data_points}")


Fetched 1500 data points, total: 1500, latest time: 2024-04-30 13:39:59.999000
Fetched 1500 data points, total: 3000, latest time: 2024-05-01 14:39:59.999000
Fetched 1500 data points, total: 4500, latest time: 2024-05-02 15:39:59.999000
Fetched 1500 data points, total: 6000, latest time: 2024-05-03 16:39:59.999000
Fetched 1500 data points, total: 7500, latest time: 2024-05-04 17:39:59.999000
Fetched 1500 data points, total: 9000, latest time: 2024-05-05 18:39:59.999000
Fetched 1500 data points, total: 10500, latest time: 2024-05-06 19:39:59.999000
Fetched 1500 data points, total: 12000, latest time: 2024-05-07 20:39:59.999000
Fetched 1500 data points, total: 13500, latest time: 2024-05-08 21:39:59.999000
Fetched 1500 data points, total: 15000, latest time: 2024-05-09 22:39:59.999000
Fetched 1500 data points, total: 16500, latest time: 2024-05-10 23:39:59.999000
Fetched 1500 data points, total: 18000, latest time: 2024-05-12 00:39:59.999000
Fetched 1500 data points, total: 19500, latest

In [5]:
len(data)

129601